# CS-E-106: Data Modeling
## Fall 2019: HW 06

In [25]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics import gofplots
import pylab


**Question 1:**

An analyst wanted to fit the regression model $Y_i = \beta_0 +\beta_1*X_{i1}+\beta_2*X_{i2}+\beta_1*X_{i3}+\epsilon_i$,
i = 1, ... , n, by the method of least squares when it is known that $\beta_2=4$. How can the analyst obtain the desired fit by using a multiple regression computer program?

**Solution 1:**

*Step 1:* Create a new variable $Y^* = Y_i-\beta_2*X_{i2} = Y_i-4*X_{i2}$

*Step 2:* Using $Y^*$ as the new response variable, run the regression model: 
$Y^* = \beta_0 +\beta_1*X_{i1}+\beta_1*X_{i3}+\epsilon_i$ using least squares (Using R functions: lm(YStar~X1+X3, data)).

*Step 3:* Use the obtained coefficients as $\beta_1$ and $\beta_3$, assuming $\beta_2=4$


**Solution 2:**

 Refer to the Commercial Properties data and problem in Assignment 5. (25 pts)

**(a)** 

Obtain the analysis of variance table that decomposes the regression sum of squares into extra sums of squares associated with X4; with X1 given X4; with X2 , given X1 and X4; and with X3 , given X1, X2 and X4. (10pts)

In [2]:
properties_data = pd.read_csv("data/Commercial Properties.csv")
lm_prop = ols("Y~X4+X1+X2+X3", data=properties_data).fit()
lm_prop.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.563   
Dependent Variable: Y                AIC:                255.4904
Date:               2019-11-11 18:00 BIC:                267.4626
No. Observations:   81               Log-Likelihood:     -122.75 
Df Model:           4                F-statistic:        26.76   
Df Residuals:       76               Prob (F-statistic): 7.27e-14
R-squared:          0.585            Scale:              1.2925  
------------------------------------------------------------------
               Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
------------------------------------------------------------------
Intercept     12.2006    0.5780  21.1099  0.0000  11.0495  13.3517
X4             0.0000    0.0000   5.7224  0.0000   0.0000   0.0000
X1            -0.1420    0.0213  -6.6549  0.0000  -0.1845  -0.0995
X2             0.2820    0.0632   4.4642  0.0000   0.1562   0.4078
X3             0.6193    1.0868   0.5699  0.5704  -1.5452   2.7839
-----------------------------------------------------------------
Omnibus:              1.922       Durbin-Watson:          1.580  
Prob(Omnibus):        0.383       Jarque-Bera (JB):       1.301  
Skew:                 0.148       Prob(JB):               0.522  
Kurtosis:             3.545       Condition No.:          1743921
=================================================================
* The condition number is large (2e+06). This might indicate
strong multicollinearity or other numerical problems.
"""

In [3]:
anova_F = anova_lm(lm_prop)
anova_F

,df,sum_sq,mean_sq,F,PR(>F)
X4,1.0,67.775098,67.775098,52.436896,3.073270e-10
X1,1.0,42.274568,42.274568,32.707399,2.003870e-07
X2,1.0,27.857493,27.857493,21.553056,1.412208e-05
X3,1.0,0.419746,0.419746,0.324753,5.704457e-01
Residual,76.0,98.230594,1.292508,NaN,NaN


In [4]:
anovaTable = pd.DataFrame(anova_F)
totals = np.array([np.sum(anovaTable["df"]), np.sum(anovaTable["sum_sq"]),np.NaN,np.NaN,np.NaN]).reshape(1,5)
totals_df = pd.DataFrame(totals, index=["Totals"], columns=anovaTable.columns)
totals_df

,df,sum_sq,mean_sq,F,PR(>F)
Totals,80.0,236.5575,NaN,NaN,NaN


In [5]:
anovaTable = anovaTable.append(totals_df)
anovaTable

,df,sum_sq,mean_sq,F,PR(>F)
X4,1.0,67.775098,67.775098,52.436896,3.073270e-10
X1,1.0,42.274568,42.274568,32.707399,2.003870e-07
X2,1.0,27.857493,27.857493,21.553056,1.412208e-05
X3,1.0,0.419746,0.419746,0.324753,5.704457e-01
Residual,76.0,98.230594,1.292508,NaN,NaN
Totals,80.0,236.557500,NaN,NaN,NaN


In [6]:
anovaTable.index = ["SSR(X4)", "SSR(X1|X4)", "SSR(X2|(X4X1))","SSR(X3|(X4X1X2))", "SSE", "Total"]

In [7]:
anovaTable

,df,sum_sq,mean_sq,F,PR(>F)
SSR(X4),1.0,67.775098,67.775098,52.436896,3.073270e-10
SSR(X1|X4),1.0,42.274568,42.274568,32.707399,2.003870e-07
SSR(X2|(X4X1)),1.0,27.857493,27.857493,21.553056,1.412208e-05
SSR(X3|(X4X1X2)),1.0,0.419746,0.419746,0.324753,5.704457e-01
SSE,76.0,98.230594,1.292508,NaN,NaN
Total,80.0,236.557500,NaN,NaN,NaN


**(b)**

From the above ANOVA table, we can see that the P-value for SSR(X3|X4X1X2) is very high, which means that the extra regression sums of squares due to X3 is very low. Thus, X3 can be dropped. F-test below.

In [8]:
ssr = anovaTable["sum_sq"]["SSR(X3|(X4X1X2))"]
sse = anovaTable["sum_sq"]["SSE"]
df_diff = 1
df_E = anovaTable["df"]["SSE"]

FStar = (ssr/df_diff) / (sse/df_E)
print(FStar)

#alpha is given
alpha = 0.01

# df from Summary above in a
FTest = stats.f.ppf(1-alpha, df_diff, df_E)
print(FTest)

0.3247533655553678
6.9805778127963585


In [9]:
p = 1-stats.f.cdf(FStar, df_diff, df_E)
print("P-value: %s" %p)

P-value: 0.5704457051158283


*Hypotheses:*

$H_0: \beta_3 = 0$ 

$H_a: \beta_3 \neq 0$ 


*Decision Rules:*

If $F^* \leq$ `r FTest`, conclude $H_0$

If $F^* >$ `r FTest`, conclude $H_a$

*Conclusion:*

Since our test statistic, $F^* =$ `r FStar`, and `r FStar` $\leq$ `r FTest`, we conclude $H_0$. Thus, X3 can be dropped from the model.


**(c)**	

Test whether both X2 and X3 can be dropped from the regression model given that X1 and X4 are retained; use α=.01. State the alternatives, decision rule, and conclusion. What is the P-value of the test? (5pts)

In [10]:
ssr = np.sum(anovaTable.loc[["SSR(X3|(X4X1X2))","SSR(X2|(X4X1))"], ["sum_sq"]])
sse = anovaTable["sum_sq"]["SSE"]
df_diff = 2
df_E = anovaTable["df"]["SSE"]

FStar = (ssr/df_diff) / (sse/df_E)
print(FStar)

#alpha is given
alpha = 0.01

# df from Summary above in a
FTest = stats.f.ppf(1-alpha, df_diff, df_E)
print(FTest)

sum_sq    10.938905
dtype: float64
4.89583988401818


In [11]:
p = 1-stats.f.cdf(FStar, df_diff, df_E)
print("P-value: %s" %p)

P-value: [6.68213643e-05]


*Hypotheses:*

$H_0: \beta_2 = \beta_3 = 0$ 

$H_a:$ Not both $\beta$s equal to zero


*Decision Rules:*

If $F^* \leq$ `r FTest`, conclude $H_0$

If $F^* >$ `r FTest`, conclude $H_a$

*Conclusion:*

Since our test statistic, $F^* =$ `r FStar`, and `r FStar` $>$ `r FTest`, we conclude $H_1$. Not both $\beta$s equal to zero.

**(d)** 

Test whether, β1 = -.1 and, β2 =.4; Use α=.01. State the alternatives, full and reduced models, decision rule, and conclusion. (5pts)

In [12]:
Y_new = properties_data["Y"]+0.1*properties_data["X1"]-0.4*properties_data["X2"]
properties_data["Y_new"] = Y_new
lm_prop_R = ols("Y_new~X3+X4", data=properties_data).fit()
lm_prop_R.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.253   
Dependent Variable: Y_new            AIC:                260.7606
Date:               2019-11-11 18:00 BIC:                267.9440
No. Observations:   81               Log-Likelihood:     -127.38 
Df Model:           2                F-statistic:        14.55   
Df Residuals:       78               Prob (F-statistic): 4.28e-06
R-squared:          0.272            Scale:              1.4121  
------------------------------------------------------------------
               Coef.   Std.Err.     t     P>|t|    [0.025   0.975]
------------------------------------------------------------------
Intercept     10.9442    0.2446  44.7371  0.0000  10.4572  11.4313
X3             2.1416    0.9906   2.1619  0.0337   0.1694   4.1138
X4             0.0000    0.0000   4.7509  0.0000   0.0000   0.0000
-----------------------------------------------------------------
Omnibus:              5.012       Durbin-Watson:          1.479  
Prob(Omnibus):        0.082       Jarque-Bera (JB):       5.926  
Skew:                 0.221       Prob(JB):               0.052  
Kurtosis:             4.249       Condition No.:          1457309
=================================================================
* The condition number is large (1e+06). This might indicate
strong multicollinearity or other numerical problems.
"""

In [13]:
anova_R = anova_lm(lm_prop_R)
anova_R

,df,sum_sq,mean_sq,F,PR(>F)
X3,1.0,9.204914,9.204914,6.518741,0.012625
X4,1.0,31.872281,31.872281,22.571327,0.000009
Residual,78.0,110.141416,1.412069,NaN,NaN


In [14]:
SSE_R = anova_R["sum_sq"]["Residual"]
SSE_F = anova_F["sum_sq"]["Residual"]
df_R = anova_R["df"]["Residual"]
df_F = anova_F["df"]["Residual"]

FStar = ((SSE_R-SSE_F)/(df_R-df_F))/(SSE_F/df_F)
print(FStar)

#alpha is given
alpha = 0.01

# df from Summary above in a
FTest = stats.f.ppf(1-alpha, (df_R-df_F), df_F)
print(FTest)

4.607640217278573
4.89583988401818


**Solution 3:**

Refer to Brand preference data and problem in Assignment 5 (30 pts)

**(a)** 

Transform the variables by means of the correlation transformation and fit the standardized regression model (10pts).

In [15]:
def standardize_corr(df):
    cols = list(df.columns)
    df_new = df.copy()
    n = df.shape[0]
    for c in cols:
        mu = np.mean(df[c])
        s = np.std(df[c])
        df_new[c] = (df[c]-mu)/(s*np.sqrt(n-1))
    
    return(df_new)

In [16]:
brand_data = pd.read_csv("data/Brand Preference.csv")
brand_data_new = standardize_corr(brand_data)
brand_data_new.describe()

,Y,X1,X2
count,1.600000e+01,1.600000e+01,16.000000
mean,2.081668e-17,-6.938894e-18,0.000000
std,2.666667e-01,2.666667e-01,0.266667
min,-4.832037e-01,-3.464102e-01,-0.258199
25%,-2.095823e-01,-1.732051e-01,-0.258199
50%,2.910866e-02,0.000000e+00,0.000000
75%,1.921171e-01,1.732051e-01,0.258199
max,4.249864e-01,3.464102e-01,0.258199


In [17]:
lm_brand_new = ols("Y~X1+X2", data=brand_data_new).fit()
lm_brand_new.summary2()

/home/srk-apts/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
==================================================================
Model:              OLS              Adj. R-squared:     0.945    
Dependent Variable: Y                AIC:                -40.5273 
Date:               2019-11-11 18:00 BIC:                -38.2095 
No. Observations:   16               Log-Likelihood:     23.264   
Df Model:           2                F-statistic:        129.1    
Df Residuals:       13               Prob (F-statistic): 2.66e-09 
R-squared:          0.952            Scale:              0.0039336
--------------------------------------------------------------------
             Coef.    Std.Err.      t      P>|t|     [0.025   0.975]
--------------------------------------------------------------------
Intercept   -0.0000     0.0157   -0.0000   1.0000   -0.0339   0.0339
X1           0.8924     0.0607   14.6952   0.0000    0.7612   1.0236
X2           0.3946     0.0607    6.4976   0.0000    0.2634   0.5258
------------------------------------------------------------------
Omnibus:               0.766        Durbin-Watson:           2.313
Prob(Omnibus):         0.682        Jarque-Bera (JB):        0.647
Skew:                  0.049        Prob(JB):                0.724
Kurtosis:              2.020        Condition No.:           4    
==================================================================

"""

$\beta_0$ is almost 0. Thus,

Regression Function: $Y = 0.892*X_1 + 0.3946*X_2$

**(b)**

Interpret the standardized regression coefficient (5pts).

*Interpretation:*

- We can see that the $\beta_0$ is almost equal to 0, which is expected since Y is now centered at 0 (based on definition and summary of the new data in part (a)).
- We also see that $\beta_1$ and $\beta_2$ are less than one and their standard deviations are also a lot less than the model without standardization.
- However, the $R^2$ is the same which means that the is equally powerful in terms of predictive capability.

**(c)**

Transform the estimated standardized regression coefficients back to the ones for the fitted regression model in the original variables (5pts).

In [20]:
lm_brand_new.params["X1"]

0.8923928635289944

In [22]:
df = brand_data

b1_star = lm_brand_new.params["X1"]
b1 = (np.std(df["Y"])/np.std(df["X1"]))*b1_star
print(b1)

b2_star = lm_brand_new.params["X2"]
b2 = (np.std(df["Y"])/np.std(df["X2"]))*b2_star
print(b2)

b0 = np.mean(df["Y"])-b1*np.mean(df["X1"])-b2*np.mean(df["X2"])
print(b0)

4.425
4.375000000000002
37.65


In [30]:
lm_brand = ols("Y~X1+X2", data=brand_data).fit()
lm_brand.params

Intercept    37.650
X1            4.425
X2            4.375
dtype: float64

**(d)**

Calculate R2Y1, R2Y2, R212, R2Y1|2, R2Y2|1 and R2. Explain what each coefficient measures and interpret your results. (10pts)

In [34]:
df = brand_data

# R2_Y1
ssr = anova_lm(ols("Y~X1", data=df).fit())["sum_sq"]["X1"]
ssto = np.sum(anova_lm(ols("Y~X1", data=df).fit()).loc[["X1","Residual"],["sum_sq"]])
R2_Y1 = ssr/ssto
print(R2_Y1)


# R2_Y2
ssr = anova_lm(ols("Y~X2", data=df).fit())["sum_sq"]["X2"]
ssto = np.sum(anova_lm(ols("Y~X2", data=df).fit()).loc[["X2","Residual"],["sum_sq"]])
R2_Y2 = ssr/ssto
print(R2_Y2)

# R2_12
R2_12 = df.corr()["X1"]["X2"]
print(R2_12)

# R2_Y1_2
ssr = anova_lm(ols("Y~X2+X1", data=df).fit())["sum_sq"]["X1"]
ssto = np.sum(anova_lm(ols("Y~X2+X1", data=df).fit()).loc[["X1","Residual"],["sum_sq"]])
R2_Y1_2 = ssr/ssto
print(R2_Y1_2)

# R2_Y2_1
ssr = anova_lm(ols("Y~X2+X1", data=df).fit())["sum_sq"]["X2"]
ssto = np.sum(anova_lm(ols("Y~X2+X1", data=df).fit()).loc[["X2","Residual"],["sum_sq"]])
R2_Y2_1 = ssr/ssto
print(R2_Y2_1)

# R2
ssr = np.sum(anova_lm(ols("Y~X1+X2", data=df).fit()).loc[["X1","X2"],["sum_sq"]])
ssto = np.sum(anova_lm(ols("Y~X1+X2", data=df).fit()).loc[["X1","X2","Residual"],["sum_sq"]])
R2 = ssr/ssto
print(R2)

sum_sq    0.796365
dtype: float64
sum_sq    0.155694
dtype: float64
0.0
sum_sq    0.943218
dtype: float64
sum_sq    0.764574
dtype: float64
sum_sq    0.952059
dtype: float64


**Solution 4:**

Refer to the CDI data set. For predicting the number of active physicians (Y) in a county, it has been decided to include total population (X1) and total personal income (X2) as predictor variables. The question now is whether an additional predictor variable would be helpful in the model and, if so, which variable would be most helpful. Assume that a first-order multiple regression model is appropriate. (25 pts)

**(a)** 

For each of the following variables, calculate the coefficient of partial determination given that X1 and X2 are included in the model: land area (X3), percent of population 65 or older (X4), number of hospital beds (X5), and total serious crimes (X6). (15pts)


In [55]:
inc_cols = ["Number of active physicians", "Total population", "Total personal income", 
             "Land area", "Percent of population 65 or older", "Number of hospital beds", 
             "Total serious crimes"]
cdi_data = pd.read_csv("data/CDI.csv")[inc_cols]
cdi_data.columns

Index(['Number of active physicians', 'Total population',
       'Total personal income', 'Land area',
       'Percent of population 65 or older', 'Number of hospital beds',
       'Total serious crimes'],
      dtype='object')

In [64]:
mapper = {
    "Number of active physicians": "Y", 
    "Total population": "X1", 
    "Total personal income":"X2", 
    "Land area": "X3", 
    "Percent of population 65 or older": "X4", 
    "Number of hospital beds": "X5",
    "Total serious crimes": "X6"
}

In [66]:
cdi_data = cdi_data.rename(columns=mapper)
cdi_data.columns

Index(['Y', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6'], dtype='object')

In [67]:
lm_cdi = ols("Y~X1+X2", data=cdi_data).fit()
lm_cdi.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                  Results: Ordinary least squares
===================================================================
Model:              OLS              Adj. R-squared:     0.899     
Dependent Variable: Y                AIC:                6832.6613 
Date:               2019-11-10 00:09 BIC:                6844.9216 
No. Observations:   440              Log-Likelihood:     -3413.3   
Df Model:           2                F-statistic:        1961.     
Df Residuals:       437              Prob (F-statistic): 5.41e-219 
R-squared:          0.900            Scale:              3.2258e+05
--------------------------------------------------------------------
               Coef.    Std.Err.     t     P>|t|     [0.025   0.975]
--------------------------------------------------------------------
Intercept     -64.4382   32.8258  -1.9630  0.0503  -128.9543  0.0779
X1              0.0005    0.0003   1.9135  0.0563    -0.0000  0.0011
X2              0.1072    0.0130   8.2692  0.0000     0.0817  0.1327
-------------------------------------------------------------------
Omnibus:             287.580       Durbin-Watson:          2.117   
Prob(Omnibus):       0.000         Jarque-Bera (JB):       3874.314
Skew:                2.611         Prob(JB):               0.000   
Kurtosis:            16.567        Condition No.:          871067  
===================================================================
* The condition number is large (9e+05). This might indicate
strong multicollinearity or other numerical problems.
"""

In [71]:
df = cdi_data

# r2_X3
ssr = anova_lm(ols("Y~X1+X2+X3", data=df).fit())["sum_sq"]["X3"]
ssto = np.sum(anova_lm(ols("Y~X1+X2+X3", data=df).fit()).loc[["X3","Residual"],["sum_sq"]])
r2_X3 = ssr/ssto
print(r2_X3)

# r2_X4
ssr = anova_lm(ols("Y~X1+X2+X4", data=df).fit())["sum_sq"]["X4"]
ssto = np.sum(anova_lm(ols("Y~X1+X2+X4", data=df).fit()).loc[["X4","Residual"],["sum_sq"]])
r2_X4 = ssr/ssto
print(r2_X4)

# r2_X5
ssr = anova_lm(ols("Y~X1+X2+X5", data=df).fit())["sum_sq"]["X5"]
ssto = np.sum(anova_lm(ols("Y~X1+X2+X5", data=df).fit()).loc[["X5","Residual"],["sum_sq"]])
r2_X5 = ssr/ssto
print(r2_X5)

# r2_X6
ssr = anova_lm(ols("Y~X1+X2+X6", data=df).fit())["sum_sq"]["X6"]
ssto = np.sum(anova_lm(ols("Y~X1+X2+X6", data=df).fit()).loc[["X6","Residual"],["sum_sq"]])
r2_X6 = ssr/ssto
print(r2_X6)

sum_sq    0.028825
dtype: float64
sum_sq    0.003842
dtype: float64
sum_sq    0.553818
dtype: float64
sum_sq    0.007323
dtype: float64


$R_{3|12}^2 =$ `r r2_X3`

$R_{4|12}^2 =$ `r r2_X4`

$R_{5|12}^2 =$ `r r2_X5`

$R_{6|12}^2 =$ `r r2_X6`

**(b)** 

On the basis of the results in part (a), which of the four additional predictor variables is best? Is the extra sum of squares associated with this variable larger than those for the other three variables? (5pts)


$X_5$ is the best predictor we can add to the model as it has the maximum coefficient of partial determination. Yes, the extra sum of squares associated with this variable is larger compared to other variables also, which makes sense since SST will remain constant.

**(c)** 

Using the F* test statistic, test whether or not the variable determined to be best in part (b) is helpful in the regression model when X1 and X2 are included in the model; use α=.01. State the alternatives, decision rule, and conclusion. Would the F* test statistics for the other three potential predictor variables be as large as the one here?  (5pts)

In [72]:
anova_X5 = anova_lm(ols("Y~X1+X2+X5", data=cdi_data).fit())
anova_X5

,df,sum_sq,mean_sq,F,PR(>F)
X1,1.0,1.243181e+09,1.243181e+09,8617.699139,2.585764e-289
X2,1.0,2.205805e+07,2.205805e+07,152.905851,2.570579e-30
X5,1.0,7.807013e+07,7.807013e+07,541.180099,2.009360e-78
Residual,436.0,6.289695e+07,1.442591e+05,NaN,NaN


In [74]:
ssr = anova_X5["sum_sq"]["X5"]
sse = anova_X5["sum_sq"]["Residual"]
df_diff = 1
df_E = anova_X5["df"]["Residual"]

FStar = (ssr/df_diff) / (sse/df_E)
print(FStar)

#alpha is given
alpha = 0.01

# df from Summary above in a
FTest = stats.f.ppf(1-alpha, df_diff, df_E)
print(FTest)

541.1800993040342
6.693357597700841


In [75]:
p = 1-stats.f.cdf(FStar, df_diff, df_E)
print("P-value: %s" %p)

P-value: 1.1102230246251565e-16


*Hypotheses:*

$H_0: \beta_5 = 0$ 

$H_a: \beta_5 \neq 0$ 


*Decision Rules:*

If $F^* \leq$ `r FTest`, conclude $H_0$

If $F^* >$ `r FTest`, conclude $H_a$

*Conclusion:*

Since our test statistic, $F^* =$ `r FStar`, and `r FStar` $\leq$ `r FTest`, we conclude $H_0$. Thus, X3 can be dropped from the model.
